In [60]:
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.functions import expr
import statsmodels.api as sm
from statsmodels.formula.api import ols, glm
from pyspark.sql.functions import isnan, when, count, col, date_format, hour
import numpy as np
from pyspark.sql import functions as F


In [18]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 2")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

In [80]:
sdf = spark.read.parquet('../data/curated/*')
zones = spark.read.csv('../data/taxi_zones/taxi+_zone_lookup.csv', \
                       header=True, inferSchema=True)
wsdf = spark.read.parquet('../data/weather_curated/*')

In [81]:
# drop irrelevant columns that we dont need to train the model on
sdf = sdf.drop('VendorID', 'passenger_count', 'trip_distance', 'RatecodeID', \
                'store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra', \
                'mta_tax', 'tip_amount', 'tolls_amount', \
                'improvement_surcharge', 'total_amount', \
                'congestion_surcharge', 'airport_fee')

## lets start doing some modelling using weather data
### we will fit linear model and see if these have an affect on weather data

In [47]:
agg_by_date_loc = (sdf.groupby(date_format('tpep_pickup_datetime', \
                                        'yyyy-MM-dd').alias('pickup_date'), \
                                        'PULocationID') \
                                    .count() \
                                    .withColumnRenamed('count', 'total_trips'))


# Joining with weather data
sdf2 = agg_by_date_loc.join(wsdf, agg_by_date_loc.pickup_date == wsdf.date, \
                            'inner').drop('pickup_date') 


In [48]:
df = sdf2.toPandas()
# Add a constant term to allow statsmodels to fit an intercept
df['const'] = 1

# Predictor variables (weather attributes + location)
predictor_vars = ['const', 'PULocationID', 'avg_wind_speed', 'precipitation', 
                  'snowfall','avg_temp'
]
X = df[predictor_vars]


# Response variable
y = df['total_trips']

model1 = sm.OLS(y, X).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:            total_trips   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     732.5
Date:                Wed, 23 Aug 2023   Prob (F-statistic):               0.00
Time:                        05:21:13   Log-Likelihood:            -8.7791e+05
No. Observations:              105003   AIC:                         1.756e+06
Df Residuals:                  104997   BIC:                         1.756e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            125.6259     12.840      9.

In [56]:
# Extract the day of the week
sdf_with_day = agg_by_date_loc.withColumn("day_of_week",\
                              date_format(col("pickup_date"), "E"))

# Create a new column that represents 1 for weekday and 0 for weekend
sdf_with_weekday = sdf_with_day.withColumn("is_weekday", \
                                            when(col("day_of_week") \
                                            .isin(["Sat", "Sun"]), 0) \
                                            .otherwise(1))
sdf3 = sdf_with_weekday.drop('day_of_week')

df = sdf3.toPandas()
# Add a constant term to allow statsmodels to fit an intercept
df['const'] = 1

# Predictor variables (weather attributes + location)
predictor_vars = ['const', 'PULocationID', 'is_weekday'
]
X = df[predictor_vars]


# Response variable
y = df['total_trips']

model2 = sm.OLS(y, X).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:            total_trips   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     1824.
Date:                Wed, 23 Aug 2023   Prob (F-statistic):               0.00
Time:                        05:35:41   Log-Likelihood:            -8.7792e+05
No. Observations:              105003   AIC:                         1.756e+06
Df Residuals:                  105000   BIC:                         1.756e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          140.5304      8.321     16.888   

In [55]:
sdf3.show(1)

+-----------+------------+-----------+----------+
|pickup_date|PULocationID|total_trips|is_weekday|
+-----------+------------+-----------+----------+
| 2022-08-02|         237|       5286|         1|
+-----------+------------+-----------+----------+
only showing top 1 row



In [58]:
sdf.show()

+--------------------+---------------------+------------+------------+----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|is_weekday|
+--------------------+---------------------+------------+------------+----------+
| 2022-08-01 00:17:39|  2022-08-01 00:19:58|         114|         148|         1|
| 2022-08-01 00:26:06|  2022-08-01 00:31:55|          79|         137|         1|
| 2022-08-01 00:45:49|  2022-08-01 00:59:29|          79|          74|         1|
| 2022-08-01 00:05:49|  2022-08-01 00:25:42|         138|         113|         1|
| 2022-08-01 00:36:29|  2022-08-01 00:51:29|         137|          68|         1|
| 2022-08-01 00:58:20|  2022-08-01 01:06:16|         132|         218|         1|
| 2022-08-01 00:22:29|  2022-08-01 00:47:32|         138|         142|         1|
| 2022-08-01 00:36:21|  2022-08-01 00:51:35|         138|         229|         1|
| 2022-08-01 00:25:17|  2022-08-01 00:39:30|         209|         236|         1|
| 2022-08-01 00:

In our analysis we saw that Manhattan has the most taxi demand but also JFK
airport not being in Manhattan but still have significant demand so in our
analysis of fitting logistic regression we will be using PU location belong to
Manhattan and JFK airport

In [82]:
filtered_zones = zones.filter(
    (F.col("Borough") == "Manhattan") | 
    (F.col("Zone") == "JFK Airport")
)

# Perform the join
result_sdf = sdf.join(
    filtered_zones, 
    sdf.PULocationID == filtered_zones.LocationID, 
    "inner"
).drop(filtered_zones.LocationID)

result_sdf.show(1, vertical = True)

-RECORD 0-------------------------------------
 tpep_pickup_datetime  | 2022-08-01 00:17:39  
 tpep_dropoff_datetime | 2022-08-01 00:19:58  
 PULocationID          | 114                  
 DOLocationID          | 148                  
 Borough               | Manhattan            
 Zone                  | Greenwich Village... 
 service_zone          | Yellow Zone          
only showing top 1 row



In [83]:

result_sdf = result_sdf.sort(F.col("tpep_pickup_datetime"))
train_sdf = result_sdf.filter((F.col("tpep_pickup_datetime") >= "2022-02-01") & 
                            (F.col("tpep_pickup_datetime") <= "2023-01-31"))

test_sdf = result_sdf.filter((F.col("tpep_pickup_datetime") >= "2023-02-01") & 
                           (F.col("tpep_pickup_datetime") <= "2023-05-31"))



In [84]:
train_sdf.show(1)

+--------------------+---------------------+------------+------------+-------+-----------+------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|Borough|       Zone|service_zone|
+--------------------+---------------------+------------+------------+-------+-----------+------------+
| 2022-02-01 00:00:00|  2022-02-01 00:15:20|         132|         258| Queens|JFK Airport|    Airports|
+--------------------+---------------------+------------+------------+-------+-----------+------------+
only showing top 1 row



In [95]:
train = train_sdf.count()
test = test_sdf.count()
print(train)
print(test)
print(f"{train/(test+train)*100}%")
train_hourly = train_sdf.withColumn("pickup_hour", hour("tpep_pickup_datetime"))
test_hourly = test_sdf.withColumn("pickup_hour", hour("tpep_pickup_datetime"))
drop_col = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'DOLocationID', 
            'Borough', 'Zone', 'service_zone'
]
train_hourly = train_hourly.drop(*drop_col)
test_hourly = test_hourly.drop(*drop_col)

35018152
11370866
75.48802175549395%


### so we have 75% of the data for training and 25% for testing

In [96]:
train_hourly.show(1)

+------------+-----------+
|PULocationID|pickup_hour|
+------------+-----------+
|         132|          0|
+------------+-----------+
only showing top 1 row

